In [2]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import xgboost as xgb
from numpy import linalg
import socket
from scipy.stats import zscore, mode
import json
import time

In [3]:
# def pm(angle):
#     if angle > np.pi:
#         while angle > np.pi:
#             angle -= 2*np.pi
#     if angle < -np.pi:
#         while angle < -np.pi:
#             angle += 2*np.pi
#     return angle

# pm = np.vectorize(pm)


# wave_dist= (299_792_458/(2.4e6+270e3))/50
# #phase_diff - 29= np.sin(np.deg2rad(angles))*2*np.pi/wave_dist
# aoa = lambda pds: np.rad2deg( np.arcsin(np.clip(pm(pds[1:] - pds[:-1] -np.deg2rad(2*99)) *wave_dist/(2*np.pi),-1,1))) 

# Classifier

In [5]:
model = xgb.XGBClassifier()
model.load_model("./tw2/pd/xgb_classifier.json")

with open("./tw2/pd/dicts.json") as file:
    dicts = json.load(file)

azi_ma = [0]*12
el_ma = [0]*12

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
    
def on_message(client, userdata, msg):
    
    data = json.loads(msg.payload)
    
    phases = np.array([np.angle(complex(data[1]["aoa"]["iq"][i],data[1]["aoa"]["iq"][i+1])) for i in range(0,len(data[1]["aoa"]["iq"]),2)])
        
    iqs = np.array([phases[i:i+4] for i in range(8,len(phases),4)][:-1])

    model_predictions = model.predict(iqs)
    
    azi = model_predictions[0::2]
    el = model_predictions[1::2]


    azi_mode = dicts["backwards_dict"][str(mode(azi).mode[0])]
    azi_ma.append(float(azi_mode))
    azi_ma.pop(0) 

    el_mode = dicts["backwards_dict"][str(mode(el).mode[0])]
    el_ma.append(float(el_mode))
    el_ma.pop(0) 

    aoa_xz = np.arctan2(np.deg2rad(np.mean(azi_ma)),np.deg2rad(np.mean(el_ma))) 

    print(f"Elevation I: {el_mode} \t Azimutal I: {azi_mode}")
    print(f"Elevation: {np.mean(el_ma)} \t Azimutal: {np.mean(azi_ma)}")
    print(f"AoA={np.rad2deg(aoa_xz)} \n")
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# client.connect("broker.hivemq.com", 1883, 60)
client.connect("test.mosquitto.org", 1883, 60)

client.loop_forever()

Conectado com o código de resultado 0


C:\Users\pedro\AppData\Local\Temp\ipykernel_14312\330458775.py:30: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  azi_mode = dicts["backwards_dict"][str(mode(azi).mode[0])]
C:\Users\pedro\AppData\Local\Temp\ipykernel_14312\330458775.py:34: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

Elevation I: -45 	 Azimutal I: -80
Elevation: -3.75 	 Azimutal: -6.666666666666667
AoA=-119.35775354279127 

Elevation I: -50 	 Azimutal I: -90
Elevation: -7.916666666666667 	 Azimutal: -14.166666666666666
AoA=-119.19748604606448 

Elevation I: -80 	 Azimutal I: -90
Elevation: -14.583333333333334 	 Azimutal: -21.666666666666668
AoA=-123.94358701975182 

Elevation I: -50 	 Azimutal I: 30
Elevation: -18.75 	 Azimutal: -19.166666666666668
AoA=-134.37040139158972 

Elevation I: -50 	 Azimutal I: 30
Elevation: -22.916666666666668 	 Azimutal: -16.666666666666668
AoA=-143.9726266148964 

Elevation I: -45 	 Azimutal I: -90
Elevation: -26.666666666666668 	 Azimutal: -24.166666666666668
AoA=-137.81555668421123 

Elevation I: -45 	 Azimutal I: -80
Elevation: -30.416666666666668 	 Azimutal: -30.833333333333332
AoA=-134.61023880468193 

Elevation I: -50 	 Azimutal I: -90
Elevation: -34.583333333333336 	 Azimutal: -38.333333333333336
AoA=-132.05595365001238 

Elevation I: -80 	 Azimutal I: -90
Eleva

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Eu ja te explico o que ta acontecendo

